In [1]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

In [ ]:
pages = {}

for page in range(200):
    
    data_page = {
        'name':[],
        'platform':[],
        'r-date':[],
        'score':[],
        'user score':[],
        'developer':[],
        'genre':[],
        'players':[],
        'critics':[],
        'users':[]
    }    
    
    # Site inside metacritic listing "Game Releases by Score"
    url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page='+str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Printing out current page
    print(50*'=', "In page: ", page)
    
    # Loop through all games in current page
    for game in soup.find_all('td', class_ = 'clamp-summary-wrap'):
        # Name
        data_page['name'].append(game.find('h3').text)
        
        print(game.find('h3').text, end="\r")
        
        # Platform
        platform = game.find('span', class_='data').text

        # Removing white space
        platform = platform.replace('\n','')
        platform = platform.replace(' ','')

        data_page['platform'].append(platform)
        
        # Release date
        data_page['r-date'].append(game.select('div.clamp-details span')[2].text)
        
        # MetaScore (has different classes depending on score)
        score_list = [
            game.find('div', class_='metascore_w large game positive'),
            game.find('div', class_='metascore_w large game mixed'),
            game.find('div', class_='metascore_w large game negative')
        ]
        
        # Filtering not none element in the score_list
        score = [s.text for s in score_list if s is not None][0]
        
        data_page['score'].append(score)
        
        # User Score (has different classes depending on score)
        score_list = [
            game.find('div', class_='metascore_w user large game positive'),
            game.find('div', class_='metascore_w user large game mixed'),
            game.find('div', class_='metascore_w user large game negative'),
            game.find('div', class_='metascore_w user large game tbd')
        ]
        
        # Filtering not none element in the score_list
        score = [s.text for s in score_list if s is not None][0]
        
        data_page['user score'].append(score)
        
        # Into the game page
        # Getting the url of the reviews page:
        url_info = game.find('a', class_='title')['href']

        url_info = 'https://www.metacritic.com'+url_info

        # Getting into the game page:
        response_info = requests.get(url_info, headers = user_agent)

        soup_info = BeautifulSoup(response_info.text, 'html.parser')

        # Get developer info

        developer = soup_info.find('li', class_ = 'summary_detail developer')
        
        if developer is not None:
            developer = developer.find('span',class_='data').text

            developer = developer.replace('\n','')
            developer = developer.replace(' ','')  

            data_page['developer'].append(developer)
        else:
            data_page['developer'].append('No info')

        # Get genre info (multiple genres are separated by commas in our entry)

        genres = soup_info.find('li', class_ = 'summary_detail product_genre')
        
        if genres is not None:
            genres = genres.find_all('span', class_='data')
            genre=''

            for item in genres:
                if genre:
                    genre = genre + ',' + item.text
                else:
                    genre = item.text

            data_page['genre'].append(genre)
        else:
            data_page['genre'].append('No info')

        # Get number of players

        players = soup_info.find('li', class_ = 'summary_detail product_players')
        
        if players is not None:
            players = players.find('span',class_='data').text
            data_page['players'].append(players)
        else:
            data_page['players'].append('No info')

        # Get number of critics

        critics = soup_info.find('div',class_='score_summary metascore_summary')
        
        if critics is not None:
            critics = critics.find('div',class_='summary').find('a').find('span').text

            if critics is not None:

                critics = critics.replace('\n','')
                critics = critics.replace(' ','')  

                data_page['critics'].append(critics)

            else:
                data_page['critics'].append('0')
        else:
            data_page['critics'].append('0')

        # get number of users

        users = soup_info.find('div',class_='details side_details')
        
        if users is not None:
            users = users.find('div',class_='score_summary')

            if users is not None:
                users = users.find('span',class_='count').find('a')

                if users is not None:
                    users = users.text
                    users = re.sub('\ Ratings$', '', users)
                    data_page['users'].append(users)
                else:
                    data_page['users'].append('0')
            else:
                data_page['users'].append('0')
        else:
            data_page['users'].append('0')
            
            
    # create a dict entry to store the dataframe for each page
    pages[str(page)] = pd.DataFrame(data_page)
    
    # export page data as csv
    pages[str(page)].to_csv('games_data-page'+str(page)+'.csv',index=False)

================================================== In page:  0
================================================== In page:  1
================================================== In page:  2
================================================== In page:  3on
================================================== In page:  4on
================================================== In page:  5's Cut
================================================== In page:  6ler
================================================== In page:  7
================================================== In page:  8
================================================== In page:  9
================================================== In page:  10
================================================== In page:  11
================================================== In page:  12
================================================== In page:  13er
================================================== In page:  14
===================================

================================================== In page:  117: Above the Law
================================================== In page:  118
================================================== In page:  119 2: Suffer the Children
================================================== In page:  120
================================================== In page:  121n'
================================================== In page:  122
================================================== In page:  123rone
================================================== In page:  124
================================================== In page:  125
================================================== In page:  126
================================================== In page:  127
================================================== In page:  128
================================================== In page:  129
================================================== In page:  130
==============================================

In [ ]:
# Create a list of all dataframes to concatenate
frames = []

for k,v in pages.items():
    frames.append(v)

In [ ]:
df_ultimate = pd.concat(frames)

In [ ]:
df_ultimate.index = range(len(df_ultimate))

In [ ]:
df_ultimate.to_csv('games-data_new.csv',index=False)

In [ ]:
df_ultimate.sort_index()
df_ultimate['r-date'] = pd.to_datetime(df_ultimate['r-date'])
df_ultimate.sort_values('r-date',inplace=True)
#df_ultimate.reset_index(inplace=True,drop=True)
#df_ultimate.drop(['level_0','index'],1)
df_ultimate

In [ ]:
df_ultimate.loc[df_ultimate['name'].str.contains("throttle", case=False)]

In [ ]:
df_ultimate.to_csv('games-data_new.csv',index=False)